# Provide geometries from CARTO with geostore IDs
This script reads a CARTO table provides each feature with a geostore ID and writes back the table to CARTO.
## Tables to read:
- [River basins - level 3](https://resourcewatch.carto.com/u/wri-rw/tables/wat_068_rw0_watersheds_edit/public?redirected=true) `SELECT * FROM "wri-rw".wat_068_rw0_watersheds_edit WHERE level = 3`

- [Country geometries - coastal](https://resourcewatch.carto.com/u/wri-rw/dataset/gadm36_0) `SELECT * FROM "wri-rw".gadm36_0 WHERE coastal = True`

- [EEZ geometries](https://resourcewatch.carto.com/u/wri-rw/tables/com_011_1_maritime_boundaries_territorial_waters/public?redirected=true): WRI still working out a few discrepancies with the country iso codes used in the two datasets. We want to ensure consistency between the code columns so a country geometry can be matched with its EEZ geometry. We plan to make some edits to the gid_0 and iso_ter1 fields by next week (May31st week), but the underlying geometries should remain the same.

## Methodology
1. Create a geojson from the CARTO table in new row
2. Get ID for geojson from Geostore in new row
3. Publish to CARTO


In [1]:
import os
from dotenv import load_dotenv
import json
import geojson
import requests
import pandas as pd
import geopandas as gpd
import cartoframes as cf
load_dotenv("../.env")
# Cartoframes docs --> https://carto.com/developers/cartoframes/reference/

True

## Functions

In [ ]:
def shpToGeojson(s):
    """Using a Shapely object, we should build a Geometry object."""
    if s.geom_type in ['Polygon', 'Point', 'MultiPoint','MultiPolygon']:
        atts={'geojson': {'type': 'FeatureCollection',
                        'features': [{'type': 'Feature',
                            'properties': {},
                            'geometry': geojson.Feature(geometry=s, properties={}).get('geometry')
                                    }]}}
        return atts
    else:
        raise ValueError('shape object was not of suitable geometry type')
    
def registerGeostore(geojson, url = 'http://api.resourcewatch.org/v1/geostore'):
        """Register valid geojson to the geostore service. Return the geostore id.
        """
        try:
            r = requests.post(url, json=geojson)
            r.raise_for_status()
            return r.json().get('data', {}).get('id','')
        except:
            return ''
        

## Authentication

In [ ]:
creds = cf.auth.Credentials(username=os.environ.get("RW_CARTO_ACCOUNT"), 
                            api_key=os.environ.get("RW_CARTO_KEY"))

## Processing

### Accessing the tables
#### Countries geometry

In [ ]:
countries_df = cf.io.carto.read_carto('SELECT * FROM "wri-rw".gadm36_0 WHERE coastal = True', credentials=creds)
countries_df.head()

In [ ]:
countries_df.info()

#### Watersheds

In [ ]:
watersheds_df = cf.io.carto.read_carto('SELECT * FROM "wri-rw".wat_068_rw0_watersheds_edit WHERE level = 3', credentials=creds)
watersheds_df.head()

#### EEZ 

In [ ]:
eez_table = 'com_011_1_maritime_boundaries_territorial_waters'
eez_df = cf.io.carto.read_carto(eez_table, credentials=creds)

eez_df.head()

### Creating geojson and getting geostore id

#### Countries geometries

In [ ]:
countries_g_id_dict = {}
countries_geojson_dict = {}
#for index, row in countries_df.iterrows():
for i in countries_df.index:
    g = countries_df.loc[[i]].the_geom.to_json()
    print(g)
    #g_id = registerGeostore(g)
    #if re.search('^Error ', g_id)!=None: #if there is an error with geostore
    #    countries_geojson_dict[row.cartodb_id] = g
    #else: 
    #    countries_g_id_dict[row.cartodb_id] = g_id
    break

In [ ]:
print(len(countries_g_id_dict))
print(len(countries_geojson_dict))
len(countries_g_id_dict)+len(countries_geojson_dict)

In [ ]:
countries_df.shape

In [ ]:
# save dicts locally
with open('countries_g_id_dict.json', 'w') as fp:
    json.dump(countries_g_id_dict, fp)
with open('countries_geojson_dict.json', 'w') as fp:
    json.dump(countries_geojson_dict, fp)

In [ ]:
countries_df.to_csv(f'./countries_df_v20210528.csv')

#### Watersheds

In [ ]:
watershed_g_id_dict = {}
watershed_geojson_dict = {}
for index, row in watersheds_df.iterrows():
    g = shpToGeojson(row.the_geom)
    g_id = registerGeostore(g)
    if re.search('^Error ', g_id)!=None: #if there is an error with geostore
        watershed_geojson_dict[row.cartodb_id] = g
    else: 
        watershed_g_id_dict[row.cartodb_id] = g_id

In [ ]:
print(len(watershed_g_id_dict))
print(len(watershed_geojson_dict))
len(watershed_g_id_dict)+len(watershed_geojson_dict)

In [ ]:
watersheds_df.shape

In [ ]:
# save dicts locally
with open('watershed_g_id_dict.json', 'w') as fp:
    json.dump(watershed_g_id_dict, fp)


In [ ]:
# transform dictionary to table with two columns and merge
watershed_id_df = pd.DataFrame.from_dict(watershed_g_id_dict, orient = 'index', columns = ["geo_id"])
watershed_id_df.reset_index(inplace=True)
watershed_id_df = watershed_id_df.rename(columns = {'index':'cartodb_id'})
watershed_id_df.merge(watersheds_df, left_on='cartodb_id', right_on='cartodb_id')
watershed_id_df.head()

In [ ]:
#save locally
watershed_id_df.to_csv(f'./watersheds_df_id_v20210528.csv')

#### EEZ

In [ ]:
eez_g_id_dict = {}
eez_geojson_dict = {}
for index, row in eez_df.iterrows():
    g = shpToGeojson(row.the_geom)
    g_id = registerGeostore(g)
    if re.search('^Error ', g_id)!=None: #if there is an error with geostore
        eez_geojson_dict[row.cartodb_id] = g
    else: 
        eez_g_id_dict[row.cartodb_id] = g_id

In [ ]:
eez_df.shape

In [ ]:
print(len(eez_g_id_dict))
print(len(eez_geojson_dict))
len(eez_g_id_dict)+len(eez_geojson_dict)

In [ ]:
# save dicts locally
with open('eez_g_id_dict.json', 'w') as fp:
    json.dump(eez_g_id_dict, fp)
with open('eez_geojson_dict.json', 'w') as fp:
    json.dump(eez_geojson_dict, fp)


In [ ]:
## save locally
eez_df.to_csv(f'./{eez_table}_v20210528.csv')

## check failed registration, `geostore_id  == ''`
- for countries
- for eez

### Countries

In [ ]:
## check failed registration, geostore_id  == ''
## s.is_valid, s.make_valid / buffer(0)

### EEZ

In [ ]:
## check failed registration, geostore_id  == ''
## s.is_valid, s.make_valid / buffer(0)

## write carto table

### Watersheds

In [ ]:
# cf.io.carto.to_carto(df, <tablename>, if_exists='replace', credentials=creds)
# cf.update_privacy_table(<tablename>, privacy='public', credentials=creds)

#### Countries

In [ ]:
# cf.io.carto.to_carto(df, <tablename>, if_exists='replace', credentials=creds)
# cf.update_privacy_table(<tablename>, privacy='public', credentials=creds)

#### EEZ

In [ ]:
# cf.io.carto.to_carto(df, <tablename>, if_exists='replace', credentials=creds)
# cf.update_privacy_table(<tablename>, privacy='public', credentials=creds)

## Normal use of carto sql API

In [1]:
url = 'https://wri-rw.carto.com/api/v2/sql'
params = {
    'q': "select * from gadm36_0 limit 1",
    'format':'geojson'
}

response = requests.get(url, params=params)

response.json()

NameError: name 'requests' is not defined

In [ ]:
sql = '''
    ALTER TABLE gadm36_0 ADD COLUMN geostore_prod VARCHAR;
'''
# ALTER TABLE gadm36_0 ADD COLUMN geostore_staging VARCHAR;
url = 'https://wri-rw.carto.com/api/v2/sql'
params = {
    'q': sql,
    'api_key': os.environ.get("RW_CARTO_KEY")
}

response = requests.get(url, params=params)

In [ ]:
response.text